In [1]:
import pandas as pd
import tensorflow as tf
import re

In [2]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [4]:
nltk.download('punkt')
# nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv("DataSets/FakeNewsDatas/train/train.csv")

In [6]:
df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [ ]:
# df['text'][0]

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df.shape

(20800, 5)

In [10]:
# So We have 20800 of records and we cant replace text data so droping 2000 null values will be good decision
# Drop Nan Values

df = df.dropna()


In [11]:
df.shape

(18285, 5)

In [12]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [13]:
# Get independent Features 
X=df.drop('label', axis=1)

In [14]:
# Get The Dependent Feature
y = df['label']

In [15]:
X.shape

(18285, 4)

In [16]:
y.shape

(18285,)

In [17]:
tf.__version__

'2.17.0'

In [18]:
# Vocabulary Size is for no. of unique words in corpus
voc_size = 5000

### OneHot Representation

In [19]:
messages=X.copy() 

In [20]:
messages['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [21]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [22]:
messages.reset_index(inplace=True)

In [23]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [24]:
stemmer = PorterStemmer()

In [25]:
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i]) # removing special char other than a,A to z,Z, 0-9 
    review = review.lower()
    words = review.split() # sent tokenization
    review = [stemmer.stem(word) for word in words if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
#     messages.loc[i,'title']= review
    corpus.append(review)
    

In [26]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [27]:
# one hot
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[4626, 4166, 1596, 2236, 4539, 4536, 3701, 635, 4712, 197],
 [965, 2554, 1996, 3915, 3623, 1749, 3817],
 [4084, 1070, 4609, 3694],
 [4833, 3512, 311, 2453, 872, 3100],
 [3338, 3623, 1129, 1066, 4053, 2462, 3623, 3492, 4302, 1369],
 [1798,
  4519,
  1597,
  2184,
  4397,
  3684,
  4114,
  1750,
  3568,
  870,
  552,
  391,
  4099,
  2655,
  3817],
 [3063, 2122, 2935, 2475, 2831, 4712, 3467, 4881, 457, 3134, 1758],
 [662, 2419, 1573, 3179, 3436, 617, 3684, 502, 457, 3134, 1758],
 [4707, 897, 1574, 3097, 1712, 212, 4407, 4171, 3684, 3717],
 [4530, 1716, 239, 2981, 4367, 989, 1029, 2612],
 [4234, 2967, 3071, 2303, 2224, 457, 538, 416, 2659, 2448, 1975],
 [2453, 4559, 4539, 212, 3684, 3436],
 [747, 1362, 988, 3773, 4754, 3730, 4120, 3548, 1618],
 [662, 3247, 4007, 1179, 1889, 4, 1523, 457, 3134, 1758],
 [880, 3830, 678, 2625, 4762, 457, 3134, 1758],
 [3892, 3243, 4793, 4705, 1289, 2978, 3631, 1719, 310, 1241],
 [1157, 4066, 2554],
 [4116, 478, 1543, 4744, 3684, 4267, 1029, 3817],
 [4016, 4

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
onehot_repr[1]

[965, 2554, 1996, 3915, 3623, 1749, 3817]

In [ ]:
# max_length = max(map(len, onehot_repr))

In [ ]:
# max_length = [len(sublist) for sublist in onehot_repr]

In [ ]:
# sorted_lst = sorted(max_length)

In [ ]:
# print(sorted_lst)

### Embedding Representation


In [ ]:
# add padding to make it same size
# Lets consider one Maximum Sentence Length

In [30]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [31]:
print(embedded_docs)

[[   0    0    0 ...  635 4712  197]
 [   0    0    0 ... 3623 1749 3817]
 [   0    0    0 ... 1070 4609 3694]
 ...
 [   0    0    0 ...  457 3134 1758]
 [   0    0    0 ... 3411 2518 4824]
 [   0    0    0 ... 1871 3424 4840]]


In [32]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4626,
       4166, 1596, 2236, 4539, 4536, 3701,  635, 4712,  197])

In [33]:
## creating model
voc_size = 5000
embedding_vector_features = 40 #Feature Representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features))
model.add(LSTM(100)) # LSTM with 100 neurons
# output is binary we will use sigmoid activation function
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [34]:
import numpy as np
X_final = np.array(embedded_docs)
y_final=np.array(y)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.7989 - loss: 0.4209 - val_accuracy: 0.9094 - val_loss: 0.2018
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9394 - loss: 0.1400 - val_accuracy: 0.9153 - val_loss: 0.1982
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9605 - loss: 0.1046 - val_accuracy: 0.9171 - val_loss: 0.2181
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9767 - loss: 0.0691 - val_accuracy: 0.9158 - val_loss: 0.2396
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9845 - loss: 0.0495 - val_accuracy: 0.9143 - val_loss: 0.2976
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9893 - loss: 0.0360 - val_accuracy: 0.9102 - val_loss: 0.3210
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9934 - loss: 0.0277 - val_accuracy: 0.9118 - val_loss: 0.3436
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9954 - loss: 0.0184 - val_accu

In [ ]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


In [38]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
confusion_matrix(y_test,y_pred)

array([[3099,  320],
       [ 202, 2414]], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9135045567522784

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

